---

# Lib

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

import re

from preprocessing.clean import vn_text_clean
from preprocessing.tokenize import vn_word_tokenize
from preprocessing.remove_stopwords import remove_stopwords

---

# Read file

In [2]:
df = pd.read_csv("../data/raw/train.csv")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

In [3]:
df.head()

,brand,model,version,year,segment,comment,label
0,hyundai,creta,n line,2025,suv,"Mẫu này đẹp và hiện đại hơn, mặt đồng hồ trung...",{EXTERIOR#Positive};{INTERIOR#Negative};
1,hyundai,creta,n line,2025,suv,Creta vẫn nên phải ở tầm giá Xforce hoặc thấp ...,{COST#Neutral};{EXTERIOR#Neutral};
2,hyundai,creta,n line,2025,suv,"Giá như cx5 dl, mặc dù con này rộng hơn và k s...",{COST#Positive};
3,hyundai,creta,n line,2025,suv,Giá cao quá đợi chương trình khuyến mại thôi,{COST#Negative};
4,hyundai,creta,n line,2025,suv,Creta là một trong những con xe mượt mà nhất c...,{EXTERIOR#Positive};{INTERIOR#Positive};{PERFO...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7786 entries, 0 to 7785
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  7786 non-null   object
 1   label    7786 non-null   object
dtypes: object(2)
memory usage: 121.8+ KB


# Clean text

In [ ]:
df["comment"] = df["comment"].apply(lambda x: vn_text_clean(x))

In [ ]:
df.head()

,comment,label
0,mới mua máy này tại thegioididong thốt nốt cảm...,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,pin kém còn lại miễn chê mua tình trạng pin cò...,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,{FEATURES#Negative};
3,mọi người cập nhật phần mềm lại nó sẽ bớt tốn ...,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,mới mua sài được tháng thấy pin rất trâu sài b...,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


In [ ]:
df["comment"] = df["comment"].apply(lambda x: vn_word_tokenize(x, method="underthesea"))

In [ ]:
df.head()

,comment,label
0,mới mua máy này tại thegioididong thốt nốt cảm...,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,pin kém còn lại miễn_chê mua tình_trạng pin cò...,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,sao lúc gọi điện_thoại màn_hình bị chấm nhỏ nh...,{FEATURES#Negative};
3,mọi người cập_nhật phần_mềm lại nó sẽ bớt tốn ...,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,mới mua sài được tháng thấy pin rất trâu sài b...,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


---

# Tokenize

---

# Remove stopwords

# Save file

In [ ]:
df.to_csv("../data/processed/train_preprocessed.csv", index=False)